In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# hello world

In [55]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [56]:
# hyperparameters
block_size = 8
batch_size = 300
lr = 3e-4
num_iter = 20000 #20000
eval_intervsl = 500
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 110
n_heads = 12
n_layers = 12
dropout = 0.2

In [57]:
torch.manual_seed(1269)

# getting the vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for i,s in enumerate(chars)}

encode = lambda s: [stoi[h] for h in s]            # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[j] for j in l])   # decoder: take a list of integers, output a string

# train and test split
data = torch.tensor(encode(text), dtype=torch.long, device=device)
n = int(0.9 * len(data))
train = data[:n]  # 90% of data
val = data[n:]    # 10% of data

In [58]:
# getting batches of data
def get_batch(split):
    data = train if split == 'train' else val
    ix =  torch.randint(len(data) - block_size, (batch_size,)) # getting random batches from the data.
    x = torch.stack([data[i: i+ block_size] for i in ix])
    y = torch.stack([data[j+1: j+block_size+1] for j in ix])
    x,y = x.to(device), y.to(device)
    return x, y

In [59]:
class Head(nn.Module):
    '''one head of self-attention.'''
    def __init__(self, head_size) -> None:
        super().__init__()
        self.query = nn.Linear( n_embd, head_size ,bias=False ,device=device)
        self.key = nn.Linear(n_embd, head_size, bias=False, device=device)
        self.value = nn.Linear(n_embd, head_size, bias=False, device=device)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size: (batch_size, time-step, channel)
        # output of size: (batch_size, time-step, head-size)
        B,T,C = x.shape
        k  = self.key(x) # (B,T,head_size)
        q = self.query(x) # (B,T,head_size)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # (B,T,T)
        wei = F.softmax(wei, dim=1)
        wei = F.dropout(wei)
        v = self.value(x)
        out = wei @ v # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)
        return out
    
class MultiHeadAttention(nn.Module):
    '''Multiple heads of self-attention running in parallel.'''
    def __init__(self, n_heads, head_size) -> None:
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.projection = nn.Linear(head_size * n_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.projection(out))
        return out
    
class FeeedForward(nn.Module):
    '''A simple linear layer followed by a non-linearity'''
    def __init__(self, n_embd) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    '''Transformer block: comunication followed by computation'''
    def __init__(self, n_embd, n_heads) -> None:
        super().__init__()
        # n_embd: embeddings dimension, n_heads: number of heads we want
        head_size = n_embd // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size)
        self.feedforward = FeeedForward(n_embd)
        self.ln1 = nn.LayerNorm( n_embd,device=device)
        self.ln2 = nn.LayerNorm(n_embd, device=device)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.feedforward(self.ln2(x))
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # each token directly reads off the logits for the next token from a look up table
        self.token_reading_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding( block_size,n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_heads=n_heads) for _ in range (n_layers)])
        self.ln_f = nn.LayerNorm(n_embd) # last layer norm
        self.lm_f = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                  torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B,T = idx.shape
        #idx and targets are both (B,T) tensors of integers
        token_embd = self.token_reading_table(idx)  # (B,T,C)
        position_embd = self.position_embedding_table(torch.arange(T, device=device))  # (T,c)
        x = position_embd + token_embd # (B,T,C)
        x = self.blocks(x)             # (B,T,C)
        x = self.ln_f(x)               # (B,T,C)
        logits = self.lm_f(x)          # (B,T,vocab_size)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last token from block_size
            idx_crop = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_crop)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # sampe from the distribution
            sample = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sample index to the running sequence
            idx = torch.cat((idx, sample), dim=1)  # (B, T+1)
        return idx


In [60]:
model = GPTLanguageModel()
m = model.to(device)
#print the number of parameters in total
print(sum(p.numel() for p in m.parameters()), 'parameters')
#create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters() ,lr=lr)

1760505 parameters


In [61]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [62]:
for iter in range(num_iter):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_intervsl == 0 or iter == num_iter - 1:
        losses = estimate_loss()
        print(f'step {iter}: train loss {losses["train"]:.4f}, vali loss {losses["val"]:.4f}')
    # sample a batch from the data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2020, vali loss 4.2000
step 500: train loss 0.5764, vali loss 0.5920
step 1000: train loss 0.3549, vali loss 0.3658
step 1500: train loss 0.3048, vali loss 0.3138
step 2000: train loss 0.2828, vali loss 0.2908
step 2500: train loss 0.2703, vali loss 0.2804
step 3000: train loss 0.2617, vali loss 0.2734
step 3500: train loss 0.2567, vali loss 0.2694
step 4000: train loss 0.2531, vali loss 0.2650
step 4500: train loss 0.2500, vali loss 0.2627
step 5000: train loss 0.2444, vali loss 0.2601
step 5500: train loss 0.2424, vali loss 0.2567
step 6000: train loss 0.2375, vali loss 0.2549
step 6500: train loss 0.2366, vali loss 0.2520
step 7000: train loss 0.2338, vali loss 0.2490
step 7500: train loss 0.2318, vali loss 0.2495
step 8000: train loss 0.2296, vali loss 0.2490
step 8500: train loss 0.2261, vali loss 0.2451
step 9000: train loss 0.2273, vali loss 0.2457
step 9500: train loss 0.2263, vali loss 0.2439
step 10000: train loss 0.2216, vali loss 0.2442
step 10500: trai

In [63]:
# generate from the model
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))


     try joy home lears;
Had me in those biding new'tck
This should, but on;
For to beforrow
Am his Evend, thought the ord us.

AELO:
For mo that prusons of madain alooks and Youl mortorouge, that? butefure!
York what I midembles a bove resumench' metham.

KING HENRY our dAance:
What is it rine, broth, though dree gulge him Edward now in a bediens; veepine.
It he wors.
But put this? for blood fair hoad:
Their should conston juson, if another: I bany wars;
I pretter the hout brother it it again! mink o' jurtive which though
To grace,
They gal.

WISTANLES:
This surt seemanant of as fawty soith ha, softer! Mayful quist him dot to doomon,
Shold, I am daught!
What mattiony's thy dest him.
Do him, pertenters, so.

PLOKENT:
Prayins of I was, if water?

ANTIUS:
Consure to his was three?
Gives, nothing this shord as common,
So the good a seat stive thou done amasted you, which it.

GLOUCES:
Soul are her every him like but, stroasol,
I know I'll us that strown.

LYORK:
To Holden. I bake he wora